## <b><font color='darkblue'>Preface</font></b>
([course link](https://learn.deeplearning.ai/courses/functions-tools-agents-langchain/lesson/2/openai-function-calling)) <b><font size='3ptx'>"Let's explore how OpenAI can extract arguments from messages and select the right function. .</font> Let's get started and unlock the potential of this powerful tool..</b>

Tool calling allows a model to respond to a given prompt by generating output that matches a user-defined schema. While the name implies that the model is performing some action, this is actually not the case! <b>The model is coming up with the arguments to a tool, and actually running the tool</b> (or not) <b>is up to the user - for example, if you want to extract output matching some schema from unstructured text, you could give the model an "extraction" tool that takes parameters matching the desired schema, then treat the generated output as your final result</b> ([more](https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/)).

<b><font color='orange'>Note</font></b>
* LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
* Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

In [1]:
!pip freeze | grep -P '(openai|langchain)'

langchain==0.2.6
langchain-anthropic==0.1.15
langchain-community==0.2.6
langchain-core==0.2.10
langchain-experimental==0.0.62
langchain-google-genai==1.0.6
langchain-groq==0.1.3
langchain-openai==0.1.9
langchain-text-splitters==0.2.0
langchainhub==0.1.14
openai==1.28.1


In [7]:
import json
import os
import openai
import re
import httpx
import os
from dotenv import load_dotenv, find_dotenv

import openai
from openai import OpenAI
from langchain_openai import ChatOpenAI

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [11]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## <b><font color='darkblue'>Define your customized functions as tools</font></b>
First, let's create a mock function `get_current_weather` to retrieve a location's temperature.  The OpenAI chatbot will use this function to provide the current weather when asked by the user.:

In [4]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [5]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

Next, let's prepare a message of user to ask about the weather:

In [6]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

Pass the message into chatbot:

In [9]:
client = OpenAI()

In [12]:
chat_completion = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    functions=functions,
)
chat_completion.choices[0].message.content

In [13]:
print(chat_completion)

ChatCompletion(id='chatcmpl-9kMlTsQtmT06hRpaFYd1EN0gv2qPB', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None))], created=1720837623, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=82, total_tokens=97))


In [18]:
chat_completion.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None)

In [19]:
chat_completion.choices[0].message.function_call

FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather')

In [20]:
chat_completion.choices[0].message.function_call.arguments

'{"location":"Boston"}'

In [25]:
args = json.loads(chat_completion.choices[0].message.function_call.arguments)
args

{'location': 'Boston'}

Let's test the chatbot's behavior by calling the function and checking if it works as expected:

In [26]:
get_current_weather(**args)

'{"location": "Boston", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

How does the chatbot handle non-weather-related questions? Let's find out:

In [27]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [28]:
chat_completion = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    functions=functions,
)
print(chat_completion)

ChatCompletion(id='chatcmpl-9kMxqYnfIUVHEoKBAeParCRzn8ADj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1720838390, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=76, total_tokens=86))


In [29]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

### <b><font color='darkgreen'>Function call approach</font></b>
There are options you can decide how the function calls are decided:
* **`auto`**:
* **`none`**:
* **`force`**:

In [31]:
help(client.chat.completions.create)

Help on method create in module openai.resources.chat.completions:

create(*, messages: 'Iterable[ChatCompletionMessageParam]', model: 'Union[str, ChatModel]', frequency_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN, function_call: 'completion_create_params.FunctionCall | NotGiven' = NOT_GIVEN, functions: 'Iterable[completion_create_params.Function] | NotGiven' = NOT_GIVEN, logit_bias: 'Optional[Dict[str, int]] | NotGiven' = NOT_GIVEN, logprobs: 'Optional[bool] | NotGiven' = NOT_GIVEN, max_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, n: 'Optional[int] | NotGiven' = NOT_GIVEN, presence_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN, response_format: 'completion_create_params.ResponseFormat | NotGiven' = NOT_GIVEN, seed: 'Optional[int] | NotGiven' = NOT_GIVEN, stop: 'Union[Optional[str], List[str]] | NotGiven' = NOT_GIVEN, stream: 'Optional[Literal[False]] | Literal[True] | NotGiven' = NOT_GIVEN, stream_options: 'Optional[ChatCompletionStreamOptionsParam] | NotGiven' = NOT_GIVE

#### <b>`function_call="auto"`</b>

In [30]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

ChatCompletion(id='chatcmpl-9kN0U4GxghkfAKwKAKgtppoR5er5W', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1720838554, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=76, total_tokens=86))


In [37]:
# We don't need to call function
response.choices[0].message.function_call is None

True

#### <b>`function_call="none"`</b>

In [40]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Taiwan?",
    }
]

response_none = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response_none)

ChatCompletion(id='chatcmpl-9kN5b7EINNXmTgshob0C2KnMvLBnW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, I'll check the current weather in Taiwan for you. One moment please.", role='assistant', function_call=None, tool_calls=None))], created=1720838871, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=83, total_tokens=100))


In [42]:
# Because we set function_call="none", it still won't call function
response.choices[0].message.function_call

#### <b>`function_call={"name": "get_current_weather"}`</b>

In [43]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-9kN6rHuiyZYhMeNUjNejHWOdke5I6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"San Francisco, CA","unit":"celsius"}', name='get_current_weather'), tool_calls=None))], created=1720838949, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=86, total_tokens=99))


In [46]:
# We can force the chatbot to call an unnecessary function, which may result in the generation of mock arguments:
response.choices[0].message.function_call

FunctionCall(arguments='{"location":"San Francisco, CA","unit":"celsius"}', name='get_current_weather')

## <b><font color='darkblue'>Supplement</font></b>
* [Deeplearning.ai - Functions, Tools and Agents with LangChain ch3: LangChain Expression Language (LCEL)](https://learn.deeplearning.ai/courses/functions-tools-agents-langchain/lesson/3/langchain-expression-language-(lcel))